### Imports

In [101]:
import numpy as np
import pandas as pd
import ace_tools_open as tools
from case_studies import *
import time
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [102]:
#Save case study functions, their derivatives and hessians in lists
fs = [f1, f4]
dfs = [df1, df4]
Hfs = [Hf1, Hf4]
fnames = ["f1", "f4"]

In [103]:
def backtracking_line_search(f, df, x, pk, alpha_init, c1, rho):
    
    alpha = alpha_init
    while f(x + alpha * pk) > f(x) + c1 * alpha * np.dot(df(x), pk):
        alpha *= rho
    return alpha

In [104]:
#Max iter afhænger nok af newton som kalder CG, måske skal det hedde noget andet
def conjugate_gradients(Q, g, eps, max_iter=1000):
    x = 0
    grad = g
    p = -grad
    counter = 0
    for _ in range(max_iter):
        Qp = np.dot(Q, p)
        alpha_k = -(np.dot(p, grad) / np.dot(p, Qp))

        x = x + alpha_k * p
        
        grad = np.dot(Q, x) + g
        
        if np.linalg.norm(grad) < eps:
            break

        p = -grad + (np.dot(grad, Qp) / np.dot(p, Qp)) * p
        grad = grad
        counter += 1
    return x, counter

In [105]:
def approximate_newton(x, f, df, hf, c1=1e-4, rho=0.9, max_iter=1000, tol=1e-6):
    xs = [x]
    cg_count_per_iter = []
    for _ in range(max_iter):

        if np.linalg.norm(df(x)) < tol:
            cg_count_per_iter.append(0)
            break 
        
        n_k = min(0.5, np.linalg.norm(df(x)))
        #n_k = 0.5 * min(0.5, np.sqrt(np.linalg.norm(df(x))))
        eps = n_k * np.linalg.norm(df(x))
        p, counter = conjugate_gradients(hf(x), df(x), eps)
        alpha = backtracking_line_search(f, df, x, p, 1.0, c1, rho)
        x = x + alpha * p
        xs.append(x)

        cg_count_per_iter.append(counter)
    return x, xs, cg_count_per_iter

In [106]:
def newtons_method(x0, f, grad_f, hessian_f, c1=1e-4, rho=0.9, tol=1e-6, max_iters=1000):
    x = x0
    xs = [x0]
    
    for _ in range(max_iters):
        grad = grad_f(x)
        hessian = hessian_f(x)
        
        if np.linalg.norm(grad) < tol:
            break 
        
        if np.all(np.linalg.eigvals(hessian) > 0):
            p = -np.linalg.solve(hessian, grad)
        else:
            eigvals, eigvecs = np.linalg.eigh(hessian)
            H = sum((1 / abs(eigval)) * np.outer(eigvec, eigvec) for eigval, eigvec in zip(eigvals, eigvecs))
            p = -H @ grad
        
        alpha = backtracking_line_search(f, grad_f, x, p, 1.0, c1, rho)
        x = x + alpha * p
        xs.append(x)
    
    return x, xs

### scipy BFGS

In [107]:
def scipy_bfgs(f,df,x0,max_iterations=1000,epsilon=1e-6):
    xs=[]
    grad_norms=[]
    def logging_f(x):
        xs.append(x)
        grad_norms.append(np.maximum(np.linalg.norm(df(x)),10**(-5)*epsilon))
        return f(x)
    minimize(logging_f, x0, method="BFGS", jac=df, tol=epsilon,options={'maxiter':max_iterations, 'gtol':epsilon})
    return np.array(xs), np.array(grad_norms)

### Strict Wolfe condition Linesearch

In [108]:
def wolfe_search(f, df, x, p, c1, c2, alpha_init):
    print("Wolf")
    def gf(alpha):
        return f(x + np.dot(alpha, p))
    def g_prime(alpha):
        return np.dot(df(x + np.dot(alpha, p)), p)
    l = 0
    u = alpha_init
    while True:
        if gf(u) > (gf(0) + c1 * u * g_prime(0)) or gf(u) > gf(l):
            break
        print("break1")
        if abs(g_prime(u)) < c2 * abs(g_prime(0)):
            alpha_star = u
            return alpha_star
        if g_prime(u) > 0:
            print("break2")
            break
        else:
            u = u * 2
    while True:
        a = (l + u) / 2
        if gf(a) > (gf(0) + c1 * a * g_prime(0)) or gf(a) > gf(l):
            print("if1")
            u = a
        else:
            if abs(g_prime(a)) < c2 * abs(g_prime(0)):
                print("if2")
                alpha_star = a
                return alpha_star
            if g_prime(a) < 0:
                print(30)
                l = a
            else:
                print(33)
                u = a

### BFGS

In [109]:
def bfgs(x, f, df, c1=1e-4, c2=0.3, max_iter=1000, tol=1e-6):
    # eye tager dimension
    H = np.eye(20)
    xs = []
    for _ in range(max_iter):
        p = -np.dot(H, df(x))
        alpha = wolfe_search(f, df, x, p, c1, c2, 1.0)
        print("wolf done")
        ap = np.dot(alpha, p)
        x_new = x + ap
        y = df(x_new) - df(x)
        s = ap
        first_term = (np.dot(s.T, y) + np.dot(y.T, np.dot(H, y)))/ ((np.dot(s.T, y))**2)
        second_term = (np.dot(H, np.outer(y,s.T))+ np.dot(np.outer(s,y.T), H))/(np.dot(s.T, y))
        H = H + first_term * np.outer(s, s) - second_term
        x = x_new
        xs.append(x)
        if np.linalg.norm(df(x)) < tol:
            break
    return x, xs

In [110]:
def benchmark(f, df, optimizer, x0, x_opt, Hf=None):
    start_time = time.time()
    if optimizer == bfgs or optimizer == scipy_bfgs:
        x_final, xs = optimizer(x0, f, df)
    if optimizer == newtons_method:
        x_final, xs = optimizer(x0, f, df, Hf)
    if optimizer == approximate_newton:
        x_final, xs, cg_count_per_iter = optimizer(x0, f, df, Hf)
    end_time = time.time()

    num_iterations = len(xs)
    final_solution_point = x_final
    dist_to_optimum = np.linalg.norm(x_final-x_opt)
    final_fun_value = f(xs[-1])
    duration = end_time-start_time
    
    grad_norms = [np.linalg.norm(df(x)) for x in xs]
    if optimizer == approximate_newton:
        return (num_iterations, duration, final_fun_value, final_solution_point, dist_to_optimum, grad_norms, grad_norms[-1], cg_count_per_iter)

    return (num_iterations, duration, final_fun_value, final_solution_point, dist_to_optimum, grad_norms, grad_norms[-1])


In [111]:
SEED = 10
np.random.seed(SEED)

d = 20
num_runs = 2

benchmark_results = []


for f, df, Hf, fname in zip(fs, dfs, Hfs, fnames):
    x_optimal = x_opt(f, d)
    #Approx Newtons Method
    for _ in range(num_runs):
        print("Approximate")
        x0 = np.random.randn(d)        
        sd_result = benchmark(f, df, approximate_newton, x0, x_optimal, Hf)
        benchmark_results.append((fname, "Approximate Newton") + sd_result)

    #Original Newtons Method
    for _ in range(num_runs):
        print("NM")
        x0 = np.random.randn(d)  
        sd_result = benchmark(f, df, newtons_method, x0, x_optimal, Hf)
        benchmark_results.append((fname, "Newtons Method") + sd_result)

    # scipy_bfgs
    # for _ in range(num_runs):
    #     x0 = np.random.randn(d)
    #     sd_result = benchmark(f, df, scipy_bfgs, x0, x_optimal)
    #     benchmark_results.append((fname, "scipy BFGS") + sd_result)

    #BFGS
    for _ in range(num_runs):
        print("BFGS")
        x0 = np.random.randn(d)
        sd_result = benchmark(f, df, bfgs, x0, x_optimal)
        benchmark_results.append((fname, "BFGS") + sd_result)
# Convert to DataFrame
columns = ["Function", "Optimizer", "Iterations", "Time", "Final Function Value", "Final Solution Point", "Distance to Optimum", "Gradient Norms", "Final Gradient Norm", "CG iterations per Newton iteration"]
df_results = pd.DataFrame(benchmark_results, columns=columns)

df_display = df_results.drop(columns=["Final Solution Point", "Gradient Norms", "CG iterations per Newton iteration"])
tools.display_dataframe_to_user(name="Benchmark Results", dataframe=df_display)

Approximate
Approximate
NM
NM
BFGS
Wolf
if1
if1
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
if1
if2
wolf done
Wolf
if1
if1
if1
33
30
if2
wolf done
Wolf
if1
if1
if1
if2
wolf done
Wolf
if1
if1
33
if2
wolf done
Wolf
if1
33
if2
wolf done
Wolf
if1
33
30
if2
wolf done
Wolf
if1
33
if2
wolf done
Wolf
if1
if2
wolf done
Wolf
33
if2
wolf done
Wolf
if1
if2
wolf done
Wolf
if2
wolf done
Wolf
break1
wolf done
Wolf
break1
wolf done
BFGS
Wolf
if1
if1
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf done
Wolf
if1
if1
if1
if1
if1
if1
if1
33
if2
wolf 